# Post Modeling

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib_inline
import joblib

from sqlalchemy import create_engine

# Matpolotlib image settings
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')


In [ ]:
# Define Functions

def run_query(query):
    """Function that takes in a query, runs it on KCMO-MC Database and returns the results in a dataframe"""

    engine = create_engine("postgresql:///kcmo-mc")
    db_conn = engine.connect()

    with db_conn.begin():
        df = pd.read_sql(query, db_conn)
    return df



def get_model_info(model_id):
    """ 
    Takes in a Model ID and returns a df with information about the model including model set id,
    model type, hyperparameters used in the model, train and validation start and end dates, etc.
    """
    
    query = f"""SELECT * from modeling.model_metadata
            LEFT JOIN  modeling.model_set_metadata 
            USING (model_set_id)
            WHERE model_id = '{model_id}'
            ;"""

    df = run_query(query)
    return df.iloc[0,:]  # Returns only first row from dataframe - all rows should be duplicates except for run id



def get_model_desc(model_id):
    """ 
    Function that takes in a Model ID and returns a brief model description, 
    including model type, model hyperparameters, vailidation end date and model id
    """
    
    model_info = get_model_info(model_id)
    model_desc = f"""{model_info['model_type']}
        {model_info['model_hyperparams']}
        validation end date: {model_info['val_end_date']}
        model_id: {model_id}"""
    return model_desc



def get_model_predictions(model_id):
    """ 
    Takes in Model ID and returns a DataFrame with the model predictions that includes
    person_id, as_of_date, score, true label and rank
    """
    query = f"""SELECT * from modeling.model_predictions
            WHERE model_id = '{model_id}'
            ;"""
    df = run_query(query)
    return df



def get_feature_importances(model_id):
    """ 
    Takes in Model ID and returns a DataFrame with feature impoortances for that model
    """
    query = f"""Select * from modeling.feature_importances
        WHERE model_id = '{model_id}'
        ;"""
    df = run_query(query)
    return df


def predictions_by_cats(model_id, crosstab_features):
    """ Takes in a Model ID and list of features of interest for crosstabs (can also be only the first words / start of the features' names).
    Returns crosstab_data, a DataFrame that includes the values of these features and model predictions 
    for each person_id; and column_names: a list that includes the column names for the features of interest
    """
    
    #Load matrix from file
    model_info = get_model_info(model_id)
    matrix = pd.read_csv(model_info['val_matrix_path'])
    #load predictions
    predictions = get_model_predictions(model_id)
    
    #get column names for all features
    column_names = []
    for f in crosstab_features:
        column = [col for col in matrix.columns if col.startswith(f)]
        column_names = column_names + column

    # join predictions and selected features
    crosstab_data = predictions.set_index('person_id').join(matrix[column_names + ['person_id']].set_index('person_id'))

    return crosstab_data, column_names



def crosstab_at_k(crosstab_data, column_names, perc = 10):
    """Function that takes in a matrix 

    Args:
        crosstab_data (DataFrame): A matrix including predictions, true labels, and features
        column_names (list): Column names for the features of interest for crosstabs
        perc (int, optional): The percentage of population to be labeled as 1 for crosstab calculations. Defaults to 10.

    Returns:
        Dataframe: Descriptives of the data according to the features listed in column_names. 
    """

    # assign top perc label 1 based on rank
    num_labeled_pos = np.ceil(perc * crosstab_data.shape[0]/100)
    crosstab_data['pred_label'] = (crosstab_data['rank'] < num_labeled_pos).astype(int)
    # assign signal detection vars
    crosstab_data['tp'] = ((crosstab_data['pred_label'] == 1) & (crosstab_data['pred_label'] == crosstab_data['true_label'])).astype(int)
    crosstab_data['fp'] = ((crosstab_data['pred_label'] == 1) & (crosstab_data['pred_label'] != crosstab_data['true_label'])).astype(int)
    crosstab_data['tn'] = ((crosstab_data['pred_label'] == 0) & (crosstab_data['pred_label'] == crosstab_data['true_label'])).astype(int)
    crosstab_data['fn'] = ((crosstab_data['pred_label'] == 0) & (crosstab_data['pred_label'] != crosstab_data['true_label'])).astype(int)

    crosstabs = pd.DataFrame()
    for column in column_names:
        col_df = pd.DataFrame()
        col_df['variable'] = [column]
        col_df['base_rate'] = crosstab_data[column].mean()
        col_df['top_k'] = (crosstab_data.loc[crosstab_data['pred_label'] == 1, column]).mean()
        for v in ['tp', 'fp', 'tn', 'fn', 'true_label']:
            mean_val = (crosstab_data.loc[crosstab_data[v] == 1, column]).mean()
            col_df[v] = [mean_val]
        crosstabs = pd.concat([crosstabs, col_df])

    return crosstabs
    


def plot_score_dist(model_id, plt_type = 1):  
    """ Takes in a Model ID and prints out the score distributions"""
    # read in scores for the selected model
    model_predictions = get_model_predictions(model_id)[['score', 'true_label']]
    #plot score distribution according to true label
    if plt_type == 2:
        sns.displot(data = model_predictions, x ='score', hue = 'true_label', kind="kde", common_norm = False, bw_method='scott')
    else:
        fig, ax = plt.subplots(1)
        sns.histplot(data = model_predictions, x ='score', hue = 'true_label', stat='probability', binwidth = .05, kde = True, ax = ax)
        ax.set_title(get_model_desc(model_id))



def plot_top_features(model_id, num_features_to_plot = 20): 
    """ Takes in a Model ID and number of to features to plot out (defaults to 20)
    """
    # read in feature importances for the model
    feature_importances = get_feature_importances(model_id)[['feature_name', 'feature_value']]
    feature_importances['abs_value'] = abs(feature_importances['feature_value'])
    feature_importances = feature_importances.sort_values(by = 'abs_value', ascending = False)
    features_to_plot = feature_importances.nlargest(num_features_to_plot,'abs_value').reset_index(drop = True)
    fig, ax = plt.subplots(1)
    sns.barplot(data = features_to_plot, y = 'feature_name', x = 'feature_value', ax = ax)
    ax.set_title(get_model_desc(model_id))



def plot_pr_curve(model_id):
    """Takes in a Model ID and plots the Precision-Recall curve"""
    # get model metrics from database
    metrics = run_query(f"SELECT * from modeling.model_metrics WHERE model_id = '{model_id}';")
    
    fig, ax = plt.subplots(1)
    # exclude AUC to get df of only precision recall
    pr_metrics =  metrics[(metrics['metric_name']=='precision') | (metrics['metric_name']=='recall')]
    pr_metrics =  pr_metrics.reset_index(drop = True)  
    sns.lineplot(data = pr_metrics, y ='value', x = 'metric_param', hue = 'metric_name', ax = ax)
    ax.set_title(get_model_desc(model_id))
    ax.set_xlabel("Percent of population labeled 1")

def get_a_metric(model_id, metric_name, metric_param):
    df = run_query(f"""SELECT * FROM modeling.model_metrics 
                       WHERE model_id = '{model_id}' 
                           and metric_name = '{metric_name}'
                           and metric_param = {metric_param}""")
    print(f"{metric_name} at {metric_param} = ", df['value'].values[0])

## Inputs

In [ ]:
# get baseline model ID
baseline_df = run_query("""SELECT * 
          FROM modeling.model_metadata
          LEFT JOIN modeling.model_set_metadata using (model_set_id)
          WHERE model_type = 'num_cases_baseline'
          ORDER BY run_id DESC, val_end_date DESC;
           """)
baseline_model_id = baseline_df.loc[0,'model_id']
baseline_model_id

In [ ]:
# insert the Model ID for the model you're interested in - otherwise this will not work

# One of the best performing models
model_id = 'c3131f37c72932bda74c61355ba65293'

# Baseline model
# model_id = baseline_model_id

# insert the percentage of population to be labeled as 1 for crosstab calculations. Defaults to 10 if no value is inserted
#perc = 10

## Precision - Recall Curve

In [ ]:
plot_pr_curve(model_id)

In [ ]:
# Calculate precision at 10 and recall at 10 
get_a_metric(model_id,'precision', 10)
get_a_metric(model_id,'recall', 10)

In [ ]:
# Calculate the percent of people who return in the bottom 10% (want to be low)
metric_param = 10
df = run_query(f"""SELECT * FROM modeling.model_predictions
                  WHERE model_id = '{model_id}';""")

val_size = df.shape[0]
num_in_bottom_10 = np.floor(val_size * metric_param/100)
num_in_bottom_10
df.loc[df['rank'] > num_in_bottom_10,'true_label'].mean()

## Score distributions

In [ ]:
plot_score_dist(model_id)

In [ ]:
plot_score_dist(model_id,2)

## Feature importance


In [ ]:
get_feature_importances(model_id)
plot_top_features(model_id)

## Cross Tabs

In [ ]:
# define the variables to split the data by
crosstab_features = ["race", "avg_age", "sex"]

In [ ]:


crosstab_data, column_names = predictions_by_cats(model_id, crosstab_features)


In [ ]:
crosstabs = crosstab_at_k(crosstab_data, column_names)
display(crosstabs)

## Bias audition

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
import yaml
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
import aequitas.plot as ap
DPI = 200


### Audit definitions


Here we define:
1) The attributes we want to audit and the reference group for each attribute
2) The metrics we are interestid in
3) Our disparity tolerance

In [ ]:
attributes_and_reference_groups={'sex':'Male', 'race':'White'}
attributes_to_audit = list(attributes_and_reference_groups.keys())
metrics = ['for', 'fnr']
disparity_tolerance = 1.30

### Load predictions, labels and attributes 

In [ ]:
#Load in data and get into the correct shape
audit_data, column_names = predictions_by_cats(model_id, attributes_to_audit)
# assign score of 1 or 1 by rank
perc = 10   # percent of defendants that will be labeled as high risk
num_labeled_pos = np.ceil(perc * audit_data.shape[0]/100)
audit_data['score'] = (audit_data['rank'] < num_labeled_pos).astype(int)

# Mapping from hot-encoding to single category columns
audit_data.loc[audit_data['sex_f'] == 1, 'sex'] = 'Female'
audit_data.loc[audit_data['sex_m'] == 1, 'sex'] = 'Male'
audit_data.loc[audit_data['sex_x'] == 1, 'sex'] = 'Trans'
audit_data.loc[audit_data['sex_missing'] == 1, 'sex'] = 'Missing'
audit_data.loc[audit_data['race_a'] == 1, 'race'] = 'Asian'
audit_data.loc[audit_data['race_b'] == 1, 'race'] = 'Black'
audit_data.loc[audit_data['race_i'] == 1, 'race'] = 'American_Indian'
audit_data.loc[audit_data['race_w'] == 1, 'race'] = 'White'
audit_data.loc[audit_data['race_u'] == 1, 'race'] = 'Unknown'
audit_data.loc[audit_data['race_missing'] == 1, 'race'] = 'Missing'

audit_data = audit_data[["score", "true_label", "sex", "race"]]
audit_data = audit_data.rename(columns = {"true_label": "label_value"})

In [ ]:
# look at the audit_data dataframe
audit_data.head()

In [ ]:
# Initialize Aequitas
g = Group()
b = Bias()

# get_crosstabs returns a dataframe of the group counts and group value bias metrics.
xtab, _ = g.get_crosstabs(audit_data, attr_cols=attributes_to_audit)
bdf = b.get_disparity_predefined_groups(xtab, original_df=audit_data, ref_groups_dict=attributes_and_reference_groups)

bdf = bdf.dropna(axis=0, how='any')

False Omission Rate (FOR) - Among people that were not classified as high risk, what is the probability of returning with a new case
as a function of race / sex?

Fase Negative Rate (FNR) - Among people that returned with a new case, what is the probability of not being classified as high risk as a function of race / sex?

In [ ]:
ap.disparity(bdf, metrics, 'race', fairness_threshold = disparity_tolerance)

In [ ]:
ap.disparity(bdf, metrics, 'sex', fairness_threshold = disparity_tolerance)

### Look at the underlying data

Disparities for all metrics:

In [ ]:
bdf[['attribute_name', 'attribute_value'] + b.list_disparities(bdf)]

All metrics:

In [ ]:
absolute_metrics = g.list_absolute_metrics(xtab)
xtab[['attribute_name', 'attribute_value'] + absolute_metrics]

Raw data counts:

In [ ]:
xtab[[col for col in xtab.columns if col not in absolute_metrics]]

In [ ]:
# Calculate overall metrics
tp = xtab['tp'].sum()
fp = xtab['fp'].sum()
tn = xtab['tn'].sum()
fn = xtab['fn'].sum()

print("Overall FOR: ", fn/(fn+tn))
print("Overall TPR: ", tp/(tp+fn))
print("Overall FPR: ", fp/(fp+tn))
